In [1]:
import os

import numpy as np
import librosa
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import load_model
from keras.utils.version_utils import callbacks
from sklearn.model_selection import StratifiedKFold, train_test_split

In [2]:
from src.utility import InstrumentDataset


def load_data():
    """ Load and preprocess data """
    x, y, classes = InstrumentDataset.read_data('../../Dataset', MERGE_FACTOR, TIME_FRAME,
                                                folder='../../Models/splits/train')
    # x, y, classes = InstrumentDataset.read_data('../../archive/NavaDataset/Data/', MERGE_FACTOR, TIME_FRAME)
    print(np.array(x).shape)
    X = np.array(x)[..., np.newaxis]  # Add an extra dimension for the channels
    print(f'The shape of X is {X.shape}')
    print(f'The shape of y is {y.shape}')
    return X, y, classes

In [3]:
TIME_FRAME = 1
MERGE_FACTOR = 5
x, y, classes = load_data()

['Tar', 'Kamancheh', 'Santur', 'Setar', 'Ney']
Tar


100%|██████████| 7646/7646 [01:34<00:00, 80.97it/s] 


Kamancheh


100%|██████████| 8039/8039 [01:40<00:00, 79.91it/s] 


Santur


...

Ney


100%|██████████| 8134/8134 [02:03<00:00, 65.97it/s] 


8640
Class: 0
Class Indices: [   0    1    2 ... 5873 5874 5875]
Class Indices Type: <class 'numpy.ndarray'>
Class Indices Dtype: int64
x Shape: (33813, 216, 64)


G:\PycharmProjects\Thesis\.venv\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=216
  warnings.warn(


Class: 1
Class Indices: [ 5876  5877  5878 ... 12490 12491 12492]
Class Indices Type: <class 'numpy.ndarray'>
Class Indices Dtype: int64
x Shape: (33813, 216, 64)
Class: 2
Class Indices: [12493 12494 12495 ... 18679 18680 18681]
Class Indices Type: <class 'numpy.ndarray'>
Class Indices Dtype: int64
x Shape: (33813, 216, 64)
Class: 3
Class Indices: [18682 18683 18684 ... 27319 27320 27321]
Class Indices Type: <class 'numpy.ndarray'>
Class Indices Dtype: int64
x Shape: (33813, 216, 64)
Class: 4
Class Indices: [27322 27323 27324 ... 33810 33811 33812]
Class Indices Type: <class 'numpy.ndarray'>
Class Indices Dtype: int64
x Shape: (33813, 216, 64)
[[-80.         -80.         -80.         ... -80.         -80.
  -80.        ]
 [-80.         -80.         -80.         ... -80.         -80.
  -80.        ]
 [-80.         -80.         -80.         ... -80.         -80.
  -80.        ]
 ...
 [-26.5782299  -22.51656914 -22.23013115 ... -37.69318008 -37.37367249
  -40.00380325]
 [-25.92939377 -22.

In [4]:
def get_encoder(x_train, y_train, x_test, y_test, batch_size, num_epochs, fold_no, input_shape, early_stopping):
    model_checkpoint_path = f'model_best_encoder_{fold_no}.keras'
    model_checkpoint_callback = ModelCheckpoint(
        filepath=model_checkpoint_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    temperature = 0.05

    layer_sizes = [512, 256, 128, 64]

    encoder = create_encoder(layer_sizes, input_shape)

    encoder_with_projection_head = add_projection_head(encoder, input_shape)
    encoder_with_projection_head.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=SupervisedContrastiveLoss(temperature),
    )

    encoder_with_projection_head.summary()

    encoder_with_projection_head.fit(
        x=x_train, y=y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=num_epochs,
        callbacks=[model_checkpoint_callback, early_stopping]
    )
    # 
    # encoder = load_model('model_best_encoder_3.keras', custom_objects={
    #     'SupervisedContrastiveLoss': SupervisedContrastiveLoss
    # }).layers[1]
    return encoder

In [5]:
from src.Instrument.Contrastive import SupervisedContrastiveLoss, create_classifier, create_encoder, \
    add_projection_head, learning_rate
from keras.callbacks import EarlyStopping
from tensorflow import keras


def train_contrastive_model(x, y, num_classes):
    """ Train Siamese network using contrastive learning """
    n_splits = 5
    if y.ndim > 1:
        y_labels = np.argmax(y, axis=1)
    else:
        y_labels = y

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
histories = []

    for fold_no, (train_index, test_index) in enumerate(skf.split(x, y_labels), start=1):
        x_train, x_test = x[train_index], x[test_index]
        # y_train, y_test = y[train_index], y[test_index]
        y_train, y_test = y_labels[train_index], y_labels[test_index]
        input_shape = (x_train.shape[1], x_train.shape[2], 1)
        model_path = f'model_best_classifier_{fold_no}.keras'
        model_callback = ModelCheckpoint(
            filepath=model_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1)
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        num_epochs = 100
        batch_size = 12

        encoder = get_encoder(x_train, y_train, x_test, y_test, batch_size, num_epochs, fold_no, input_shape,
                              early_stopping)

        classifier = create_classifier(encoder, num_classes, input_shape, trainable=False)

        classifier.summary()

        history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs,
                                 validation_data=(x_test, y_test), callbacks=[model_callback, early_stopping])

        accuracy = classifier.evaluate(x_test, y_test)[1]
        print(f"Test accuracy: {round(accuracy * 100, 2)}%")

        histories.append(history)
        fold_no += 1

    return histories


G:\PycharmProjects\Thesis\.venv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
G:\PycharmProjects\Thesis\.venv\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want t

In [ ]:
histories = train_contrastive_model(x, y, len(classes))

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 216, 64, 1)]      0         
                                                                 
 sequential (Sequential)     (None, 256)               3524544   
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
Total params: 3,557,440
Trainable params: 3,555,520
Non-trainable params: 1,920
_________________________________________________________________
Epoch 1/100
1366/2880 [=============>................] - ETA: 1:41 - loss: 1.4607